# 模型训练

## Step1 导包

In [1]:
import numpy as np
import tiktoken
import torch 
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset , DataLoader
import matplotlib 
from datasets import load_dataset
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler

g:\Anaconda\envs\llms4scratch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 模型初始化

In [2]:
from model import GPTModel
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 1024, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "dropout": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

In [3]:
model = GPTModel(GPT_CONFIG_124M)
tokenizer = tiktoken.get_encoding("gpt2")

## Step3 加载数据集

In [4]:
import os
import urllib.request

file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
#引入数据集
if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode('utf-8')
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()
#一系列经典的读取数据操作

In [10]:
print(len(text_data))
print(text_data[-99:])

20479
it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."


## Step4 处理数据集

In [5]:
encoded_token = tokenizer.encode(text_data)
print(len(encoded_token))

5145


In [6]:
class GPTDataset(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        super().__init__()
        self.input_ids = []
        self.target_ids = []

        #生成encoded
        encoded = tokenizer.encode(txt)

        #生成数据
        for i in range(0,len(encoded)-max_length,stride):
            input = encoded[i:i+max_length]
            target = encoded[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input))
            self.target_ids.append(torch.tensor(target))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, index):
        return self.input_ids[index],self.target_ids[index]

In [7]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDataset(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [12]:
train_ratio = 0.8
train_sample = int(train_ratio * len(text_data))
train_data = text_data[:train_sample]
test_data = text_data[train_sample:]
train_loader = create_dataloader_v1(
    train_data,
    batch_size = 4,
    max_length=GPT_CONFIG_124M["context_length"],
    stride = 128,
    shuffle = True,
    drop_last = True,
    num_workers = 0
)
test_loader = create_dataloader_v1(
    test_data,
    batch_size = 4,
    max_length=GPT_CONFIG_124M["context_length"],
    stride = 128,
    shuffle = False,
    drop_last = False,
    num_workers = 0
)

In [13]:
if len(encoded_token)* (train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the training loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "increase the `training_ratio`")

if len(encoded_token) * (1-train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the validation loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "decrease the `training_ratio`")

## Step5 构建训练函数

### 损失函数

In [14]:

def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    #呼唤GPU
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    #用交叉熵函数对于logits进行计算并且拉伸到二维长度
    return loss
#一个计算批损失的函数

def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # 如果指定的批次数超过数据加载器中的总批次数，则将批次数减少到与数据加载器的总批次数匹配。
        num_batches = min(num_batches, len(data_loader))
        #减少需要处理的数量,同时也是防止溢出
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
        #一点点加上去的损失
    return total_loss / num_batches

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 如果支持，则调用 GPU

# 注意：
# 如果取消注释以下代码块，代码可以在 Apple Silicon 芯片上运行（如果适用），
# 在 M3 MacBook Air 上测量速度大约是 Apple CPU 的两倍。
# 然而，计算得到的损失值可能会略有不同。

#if torch.cuda.is_available():
#    device = torch.device("cuda")
#elif torch.backends.mps.is_available():
#    device = torch.device("mps")
#else:
#    device = torch.device("cpu")
#
# print(f"Using {device} device.")

model.to(device)  # 对于 nn.Module 类，不需要赋值 model = model.to(device)

torch.manual_seed(123)  # 固定随机种子，保证数据加载器打乱数据的结果可复现

with torch.no_grad():  # 禁用梯度跟踪以提高效率，因为此时尚未开始训练
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(test_loader, model, device)

# 推理阶段不计算梯度

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 11.003416220347086
Validation loss: 11.027277946472168


In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1
    #初始化训练模型而且给了空的队列
    # Main training loop
    for epoch in range(num_epochs):#训练次数
        model.train()  # Set model to training mode
        #转移到训练模块
        for input_batch, target_batch in train_loader:
            #从loader里面调出输入跟目标
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            #清空所有函数的梯度
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            #计算损失函数
            loss.backward() # Calculate loss gradients
            #反向传播优化
            optimizer.step() # Update model weights using loss gradients
            #更新权重
            tokens_seen += input_batch.numel()
            #加一下一共有多少
            global_step += 1
            #看一下一共训练了多少步
            # Optional evaluation step
            if global_step % eval_freq == 0:
                #按照一定的步数进行记录
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                #计算损失函数
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                #加到list中
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    #评价模块
    model.eval()
    #检验模式
    with torch.no_grad():
        #我认为的双保险,防止梯度更新
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    #	在评估结束后切换回训练模式，确保模型能继续用于训练。
    return train_loss, val_loss


In [ ]:
# Note:
# Uncomment the following code to calculate the execution time
#下面可以看一下计算了多久
# import time
# start_time = time.time()

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
#经典操作
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
#用Adam进行优化,其中学习rate为0.004,动量衰减是0.1
num_epochs = 1
#10论学习
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, test_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

In [ ]:

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots(figsize=(5, 3))

    # Plot training and validation loss against epochs
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(epochs_seen, val_losses, linestyle="-.", label="Validation loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))  # only show integer labels on x-axis

    # Create a second x-axis for tokens seen
    ax2 = ax1.twiny()  # Create a second x-axis that shares the same y-axis
    ax2.plot(tokens_seen, train_losses, alpha=0)  # Invisible plot for aligning ticks
    ax2.set_xlabel("Tokens seen")

    fig.tight_layout()  # Adjust layout to make room
    plt.savefig("loss-plot.pdf")
    plt.show()

epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)
#一个经典的plot画图函数

##  Step6 测试模型性能

In [ ]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_token, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Get the idx of the vocab entry with the highest logits value
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [ ]:
print(tokenizer.encode("我是无敌jjy"))

In [ ]:

model.to("cpu")
model.eval()

tokenizer = tiktoken.get_encoding("gpt2")
start_context = "你好，我是无敌jjy"
encoded = tokenizer.encode(start_context)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)

print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
print("\nInput text:", start_context)
print("Encoded input text:", encoded)
print("encoded_tensor.shape:", encoded_tensor.shape)

out = generate_text_simple(
        model=model,
        idx=encoded_tensor,
        max_new_tokens=10,
        context_size=GPT_CONFIG_124M["context_length"]
    )
decoded_text = tokenizer.decode(out.squeeze(0).tolist())

print(f"\n\n{50*'='}\n{22*' '}OUT\n{50*'='}")
print("\nOutput:", out)
print("Output length:", len(out[0]))
print("Output text:", decoded_text)